# Fedbiomed Researcher to train a model on a CSV dataset

Use for developing (autoreloads changes made across packages)

In [1]:
%load_ext autoreload
%autoreload 2

This example shows how to use a CSV format file as a node dataset. The example CSV file is synthetic data with a format inspired from ADNI dataset.

## Start the network
Before running this notebook, start the network with `./scripts/fedbiomed_run network`

## Setting the node up
It is necessary to previously configure a node:
1. `./scripts/fedbiomed_run node add`
  * Select option 1 to add a csv file to the node
  * Choose the name, tags and description of the dataset
    * use `#test_data` for the tags
  * Pick the .csv file from your PC (here: [pseudo_adni_mod.csv](./data/CSV/pseudo_adni_mod.csv))
  * Data must have been added
2. Check that your data has been added by executing `./scripts/fedbiomed_run node list`
3. Run the node using `./scripts/fedbiomed_run node start`. Wait until you get `Starting task manager`. it means you are online.

## Create an experiment to train a model on the data found

Declare a torch.nn MyTrainingPlan class to send for training on the node

In [2]:
import torch
import torch.nn as nn
from fedbiomed.common.training_plans import TorchTrainingPlan
from fedbiomed.common.data import DataManager
from torch.utils.data import Dataset
import pandas as pd

# Here we define the model to be used. 
# You can use any class name (here 'MyTrainingPlan')
class MyTrainingPlan(TorchTrainingPlan):
    def __init__(self, model_args: dict = {}):
        super(MyTrainingPlan, self).__init__(model_args)
        # should match the model arguments dict passed below to the experiment class
        self.in_features = model_args['in_features']
        self.out_features = model_args['out_features']
        self.fc1 = nn.Linear(self.in_features, 5)
        self.fc2 = nn.Linear(5, self.out_features)
        
        # Here we define the custom dependencies that will be needed by our custom Dataloader
        # In this case, we need the torch Dataset and DataLoader classes
        # We need pandas to read the local .csv file at the node side
        deps = ["from torch.utils.data import Dataset, DataLoader",
                "import pandas as pd"]
        self.add_dependency(deps)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

    def training_step(self, data, target):
        output = self.forward(data).float()
        criterion = torch.nn.MSELoss()
        loss   = criterion(output, target.unsqueeze(1))
        return loss

      # OLD WAY 
#     class csv_Dataset(Dataset):
#     # Here we define a custom Dataset class inherited from the general torch Dataset class
#     # This class takes as argument a .csv file path and creates a torch Dataset 
#         def __init__(self, dataset_path, x_dim):
#             self.input_file = pd.read_csv(dataset_path,sep=';',index_col=False)
#             x_train = self.input_file.iloc[:,:x_dim].values
#             y_train = self.input_file.iloc[:,-1].values
#             self.X_train = torch.from_numpy(x_train).float()
#             self.Y_train = torch.from_numpy(y_train).float()

#         def __len__(self):            
#             return len(self.Y_train)

#         def __getitem__(self, idx):

#             return (self.X_train[idx], self.Y_train[idx])
        
    def training_data(self,  batch_size = 48):
        # OLD WAY
        # The training_data creates the Dataloader to be used for training in the 
        # general class TorchTrainingPlan of fedbiomed
        # dataset = self.csv_Dataset(self.dataset_path, self.in_features)
        
        # NEW WAY
        # Training data method reads the dataset and craete DataManager by passing 
        # input variables and output(target) variable. DataManager will automaticly 
        # create PyTorch Dataset instnce for training
        
        df = pd.read_csv(self.dataset_path, sep=';', index_col=False)
        x_dim = self.in_features
        x_train = df.iloc[:,:x_dim].values
        y_train = df.iloc[:,-1].values
        train_kwargs = {'batch_size': batch_size, 'shuffle': True}
        
        data_manager = DataManager(dataset=x_train , target=y_train, **train_kwargs)
        
        return data_manager

In [3]:
# model parameters 
model_args = {
    'in_features': 15, 
    'out_features': 1
}

# training parameters 
training_args = {
    'batch_size': 20, 
    'lr': 1e-3, 
    'epochs': 10, 
    'dry_run': False,  
    #'batch_maxnum': 100 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}

Define an experiment
- search nodes serving data for these `tags`, optionally filter on a list of node ID with `nodes`
- run a round of local training on nodes with model defined in `model_path` + federation with `aggregator`
- run for `round_limit` rounds, applying the `node_selection_strategy` between the rounds

In [4]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

# Calling the training data with specified tags
tags =  ['#test_data']
rounds = 5

exp = Experiment(tags=tags,
                 model_class=MyTrainingPlan,
                 model_args=model_args,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None)

2022-03-08 13:23:36,749 fedbiomed INFO - Component environment:
2022-03-08 13:23:36,749 fedbiomed INFO - type = ComponentType.RESEARCHER
2022-03-08 13:23:36,913 fedbiomed INFO - Messaging researcher_420cfc13-37cb-447c-af20-f7ac5cb2b6ab successfully connected to the message broker, object = <fedbiomed.common.messaging.Messaging object at 0x7f403823f8e0>
2022-03-08 13:23:36,930 fedbiomed INFO - Searching dataset with data tags: ['#test_data'] for all nodes
2022-03-08 13:23:36,932 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / DEBUG - Message received: {'researcher_id': 'researcher_420cfc13-37cb-447c-af20-f7ac5cb2b6ab', 'tags': ['#test_data'], 'command': 'search'}
2022-03-08 13:23:46,967 fedbiomed INFO - Node selected for training -> node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0
2022-03-08 13:23:46,996 fedbiomed DEBUG - Model file has been saved: /home/scansiz/Desktop/Inria/development/fedbiomed/var/experiments/Experiment_0078/my_model_ac67c1ff-8bed-46c2-924a-1d5680ede

Let's start the experiment.

By default, this function doesn't stop until all the `round_limit` rounds are done for all the nodes

In [5]:
exp.run()

2022-03-08 13:23:48,258 fedbiomed INFO - Sampled nodes in round 0 ['node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0']
2022-03-08 13:23:48,259 fedbiomed INFO - Send message to node node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 - {'researcher_id': 'researcher_420cfc13-37cb-447c-af20-f7ac5cb2b6ab', 'job_id': '475f221e-15f7-4b50-959f-daa638099ada', 'training_args': {'batch_size': 20, 'lr': 0.001, 'epochs': 10, 'dry_run': False}, 'model_args': {'in_features': 15, 'out_features': 1}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/03/08/my_model_ac67c1ff-8bed-46c2-924a-1d5680edecb5.py', 'params_url': 'http://localhost:8844/media/uploads/2022/03/08/aggregated_params_init_bf9d8ed1-2377-4f3d-a2cc-45c8cc93cff8.pt', 'model_class': 'MyTrainingPlan', 'training_data': {'node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0': ['dataset_499de3c8-f308-432f-b284-9b90a52df828']}}
2022-03-08 13:23:48,260 fedbiomed DEBUG - researcher_420cfc13-37cb-447c-af20-f7ac5cb2b6ab
2022-03-08 13:23:48,263 fedbiom

2022-03-08 13:23:48,998 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 7 [0/1000 (0%)]	Loss: 117.601181
2022-03-08 13:23:49,011 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 7 [200/1000 (20%)]	Loss: 208.652985
2022-03-08 13:23:49,022 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 7 [400/1000 (40%)]	Loss: 181.314270
2022-03-08 13:23:49,032 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 7 [600/1000 (60%)]	Loss: 203.751434
2022-03-08 13:23:49,041 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 7 [800/1000 (80%)]	Loss: 260.507080
2022-03-08 13:23:49,050 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 8 [0/1000 (0%)]	Loss: 173.732925
2022-03-08 13:23:49,060 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO 

2022-03-08 13:23:58,354 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / DEBUG - Using device cpu for training (cuda_available=False, gpu=False, gpu_only=False, use_gpu=False, gpu_num=None)
2022-03-08 13:23:58,356 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 1 [0/1000 (0%)]	Loss: 99.859169
2022-03-08 13:23:58,366 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 1 [200/1000 (20%)]	Loss: 138.696533
2022-03-08 13:23:58,378 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 1 [400/1000 (40%)]	Loss: 159.520020
2022-03-08 13:23:58,392 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 1 [600/1000 (60%)]	Loss: 166.616333
2022-03-08 13:23:58,403 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 1 [800/1000 (80%)]	Loss: 179.313171
2022-03-08 13:23:58,417 fedbiomed INF

2022-03-08 13:23:58,916 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - results uploaded successfully 
2022-03-08 13:24:08,337 fedbiomed INFO - Downloading model params after training on node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 - from http://localhost:8844/media/uploads/2022/03/08/node_params_a75d2534-9aae-417c-99c0-8eb326d1a9dd.pt
2022-03-08 13:24:08,342 fedbiomed DEBUG - upload (HTTP GET request) of file node_params_d2093d55-94d5-45df-9e60-b14f0d3a25df.pt successful, with status code 200
2022-03-08 13:24:08,344 fedbiomed INFO - Nodes that successfully reply in round 1 ['node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0']
2022-03-08 13:24:08,391 fedbiomed DEBUG - upload (HTTP POST request) of file /home/scansiz/Desktop/Inria/development/fedbiomed/var/experiments/Experiment_0078/aggregated_params_484bc836-63a2-4fd6-b11d-cc67126c639f.pt successful, with status code 201
2022-03-08 13:24:08,392 fedbiomed INFO - Saved aggregated params for round 1 in /home/scansiz/De

2022-03-08 13:24:08,686 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 5 [400/1000 (40%)]	Loss: 91.933914
2022-03-08 13:24:08,696 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 5 [600/1000 (60%)]	Loss: 89.657372
2022-03-08 13:24:08,707 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 5 [800/1000 (80%)]	Loss: 87.117050
2022-03-08 13:24:08,717 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 6 [0/1000 (0%)]	Loss: 135.761276
2022-03-08 13:24:08,727 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 6 [200/1000 (20%)]	Loss: 151.633942
2022-03-08 13:24:08,737 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 6 [400/1000 (40%)]	Loss: 149.142929
2022-03-08 13:24:08,747 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO 

...

2022-03-08 13:31:44,017 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 2 [7200/10668 (67%)]	Loss: 16211.733398
2022-03-08 13:31:44,028 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 2 [7600/10668 (71%)]	Loss: 13141.957031
2022-03-08 13:31:44,042 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 2 [8000/10668 (75%)]	Loss: 15203.846680
2022-03-08 13:31:44,056 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 2 [8400/10668 (79%)]	Loss: 31231.083984
2022-03-08 13:31:44,068 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 2 [8800/10668 (82%)]	Loss: 15301.529297
2022-03-08 13:31:44,082 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 2 [9200/10668 (86%)]	Loss: 12253.433594
2022-03-08 13:31:44,096 fedbiomed INFO - log from: node_bd90cc6f-67c4-

2022-03-08 13:31:53,847 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 1 [8800/10668 (82%)]	Loss: 22081.679688
2022-03-08 13:31:53,868 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 1 [9200/10668 (86%)]	Loss: 10967.745117
2022-03-08 13:31:53,885 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 1 [9600/10668 (90%)]	Loss: 9250.675781
2022-03-08 13:31:53,902 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 1 [10000/10668 (94%)]	Loss: 21695.341797
2022-03-08 13:31:53,919 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 1 [10400/10668 (97%)]	Loss: 10472.600586
2022-03-08 13:31:53,936 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 2 [0/10668 (0%)]	Loss: 13563.573242
2022-03-08 13:31:53,953 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f8

2022-03-08 13:38:51,449 fedbiomed INFO - log from: node_1234 / INFO - Recreating hashing for : variational-autoencoder.txt 	 model_d7d1e03b-f666-4328-9218-15ece72806da
2022-03-08 13:38:51,471 fedbiomed INFO - log from: node_1234 / INFO - Recreating hashing for : sklearn-perceptron.txt 	 model_c20a3130-4534-4262-a7ec-488e417547db
2022-03-08 13:38:51,472 fedbiomed INFO - log from: node_1234 / INFO - Recreating hashing for : pytorch-csv.txt 	 model_76b917c0-da54-49eb-a90c-43ccaa6b99e2
2022-03-08 13:38:51,489 fedbiomed INFO - log from: node_1234 / INFO - Recreating hashing for : sklearn-sgdregressor.txt 	 model_b3b5e5b3-0c3d-4dcd-9424-f797dff7122f
2022-03-08 13:38:51,491 fedbiomed INFO - log from: node_1234 / INFO - Recreating hashing for : pytorch-mnist.txt 	 model_951063d1-14e2-4998-8f16-aeda885e13da
2022-03-08 13:38:51,503 fedbiomed INFO - log from: node_1234 / INFO - Recreating hashing for : pytorch-celaba.txt 	 model_56eaa8c1-fb07-474e-8fe6-dc54d2136c61
2022-03-08 13:38:51,531 fedbiom

2022-03-08 13:38:51,900 fedbiomed INFO - log from: node_1234 / INFO - Recreating hashing for : variational-autoencoder.txt 	 model_d7d1e03b-f666-4328-9218-15ece72806da
2022-03-08 13:38:51,915 fedbiomed INFO - log from: node_1234 / INFO - Recreating hashing for : sklearn-perceptron.txt 	 model_c20a3130-4534-4262-a7ec-488e417547db
2022-03-08 13:38:51,926 fedbiomed INFO - log from: node_1234 / INFO - Recreating hashing for : pytorch-csv.txt 	 model_76b917c0-da54-49eb-a90c-43ccaa6b99e2
2022-03-08 13:38:51,938 fedbiomed INFO - log from: node_1234 / INFO - Recreating hashing for : sklearn-sgdregressor.txt 	 model_b3b5e5b3-0c3d-4dcd-9424-f797dff7122f
2022-03-08 13:38:51,952 fedbiomed INFO - log from: node_1234 / INFO - Recreating hashing for : pytorch-mnist.txt 	 model_951063d1-14e2-4998-8f16-aeda885e13da
2022-03-08 13:38:51,964 fedbiomed INFO - log from: node_1234 / INFO - Recreating hashing for : pytorch-celaba.txt 	 model_56eaa8c1-fb07-474e-8fe6-dc54d2136c61
2022-03-08 13:38:51,986 fedbiom

2022-03-08 13:38:52,848 fedbiomed INFO - log from: node_1234 / DEBUG - Details of exception: mimicking an HTTP error
2022-03-08 13:38:52,849 fedbiomed INFO - log from: node_1234 / ERROR - FB201: server not reachable : CALLABLE_RAISING_TIMEOUT_EXCEPTION HTTP request time exceeds Timeout
2022-03-08 13:38:52,850 fedbiomed INFO - log from: node_1234 / ERROR - FB201: server not reachable : CALLABLE_RAISING_TOO_MANY_REDIRECTION_ERRORS HTTP request exceeds max number of redirection
2022-03-08 13:38:52,851 fedbiomed INFO - log from: node_1234 / ERROR - FB604: repository error : bad URL when issuing a unknwon HTTP request a/file/path(details :Invalid URL 'a/file/to/upload': No scheme supplied. Perhaps you meant http://a/file/to/upload? )
2022-03-08 13:38:53,413 fedbiomed INFO - log from: node_1234 / ERROR - FB201: server not reachable when issuing a unknwon HTTP request a/file/path to http://a.fake.url: name or service not known
2022-03-08 13:38:53,417 fedbiomed INFO - log from: node_1234 / ERR

Local training results for each round and each node are available via `exp.training_replies()` (index 0 to (`rounds` - 1) ).

For example you can view the training results for the last round below.

Different timings (in seconds) are reported for each dataset of a node participating in a round :
- `rtime_training` real time (clock time) spent in the training function on the node
- `ptime_training` process time (user and system CPU) spent in the training function on the node
- `rtime_total` real time (clock time) spent in the researcher between sending the request and handling the response, at the `Job()` layer

In [ ]:
print("\nList the training rounds : ", exp.training_replies().keys())

print("\nList the nodes for the last training round and their timings : ")
round_data = exp.training_replies()[rounds - 1].data()
for c in range(len(round_data)):
    print("\t- {id} :\
    \n\t\trtime_training={rtraining:.2f} seconds\
    \n\t\tptime_training={ptraining:.2f} seconds\
    \n\t\trtime_total={rtotal:.2f} seconds".format(id = round_data[c]['node_id'],
        rtraining = round_data[c]['timing']['rtime_training'],
        ptraining = round_data[c]['timing']['ptime_training'],
        rtotal = round_data[c]['timing']['rtime_total']))
print('\n')
    
exp.training_replies()[rounds - 1].dataframe()

Federated parameters for each round are available in `exp.aggregated_params()` (index 0 to (`rounds` - 1) ).

For example you can view the federated parameters for the last round of the experiment :

In [ ]:
print("\nList the training rounds : ", exp.aggregated_params().keys())

print("\nAccess the federated params for the last training round :")
print("\t- params_path: ", exp.aggregated_params()[rounds - 1]['params_path'])
print("\t- parameter data: ", exp.aggregated_params()[rounds - 1]['params'].keys())
